# Example for Financial Instruments

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../..') 

import assets 
import numpy as np
from datetime import datetime, timedelta

### We can create financial instruments like stocks, currencies, futures and options

In [2]:
stock = assets.Stock(ticker='SPY', price=600)
print(f"Stock's ticker: {stock.name} and stock's price {stock.price}")

Stock's ticker: SPY and stock's price 600


### Note that you cannot create more than one object of the same class with the same name. The idea is that these classes represent the asset class of the financial instruments, not a particular position.

In [3]:
stock_duplicate = assets.Stock(ticker=stock.name)
stock_duplicate is stock

True

### However the price is updated to the last object declaration. Here, since no price was given, it defaults to `None`:

In [4]:
stock.price is None

True

### Another way to change the price of an Asset is via the method `Asset.set_price(price)`. On the other hand, setting `Asset.price = price` would yield an `AttributeError`. Trying to change the name `Asset.name` would also yield the same error, just that `Asset.name` is immutable, i.e. there is no method to change it, since it would change the identity of the asset.

In [5]:
stock.set_price(605)
stock.price

605

### Now we define a call option on our stock. For that we will use the expiration in exactly one year of today (or whatever day on which you are executing the cell)

In [6]:
one_year_later = datetime.now() + timedelta(days=365)
expiration_str = one_year_later.strftime("%y%m%d")

option = assets.Option(underlying=stock, strike=605, expiration=expiration_str, option_type="C", price=200)

### We can now corroborate that expiration lies 1 yr from now, and that the option has not expired: `option.expiration.is_expired() = False`

In [7]:
print(f"Time till expiration: {option.expiration.T} yr. Has the option expired? -> {option.expiration.is_expired()}")

Time till expiration: 1.0012275177574834 yr. Has the option expired? -> False


### We can check what the price of the option would be at the time of expiration, given the spot price `ST` of our underlying.

In [8]:
ST = np.arange(600, 610, 2) # Spot price
ST, option.price_at_expiration(ST)

(array([600, 602, 604, 606, 608]), array([  0,   0,   0, 100, 300]))

### Since expiration will always be computed dynamically, the time till expiration will change every second that passes by. To give the possibility to fix the time, there is a method `ExpirationDate.fix_time(T)`, where `T` is the time in years till expiration that should be keep static. For example,

In [9]:
option_static = assets.Option(underlying=stock, strike=605, expiration="250101", option_type="C")

if option_static.expiration.is_expired():
    print(f"The asset {option_static} has already expired.")
else:
    print(f"The asset {option_static} has NOT expired.")

The asset Option(SPY250101C00605000) has already expired.


### Note that this option has already expired, since `expiration` lies in the past. However, in order to fix the time, we could write `option_static.expiration.fix_time(T=0.5)` and this will take us 6 months before the expiration s.t. `option_static.expiration.T` is `0.5`, and the option has not yet expired. This will remain so, unless we unfix the time with `ExpirationDate.unfix_time()`

In [10]:
option_static.expiration.fix_time(0.5)

if option_static.expiration.is_expired():
    print(f"The asset {option_static} has already expired.")
else:
    print(f"The asset {option_static} has NOT expired.")
    
option_static.expiration.T == 0.5

The asset Option(SPY250101C00605000) has NOT expired.


True

In [11]:
option_static.expiration.unfix_time()

if option_static.expiration.is_expired():
    print(f"The asset {option_static} has already expired.")
else:
    print(f"The asset {option_static} has NOT expired.")

option_static.expiration.expiration_date

The asset Option(SPY250101C00605000) has already expired.


'250101'